In [1]:
import pandas as pd

In [2]:
ucs_sats_messy = pd.read_csv('../data/original/UCS-Satellite-Database 5-1-2023.csv')

ucs_sats_messy.head(10)

,"Name of Satellite, Alternate Names",Current Official Name of Satellite,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,...,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67
0,1HOPSAT-TD (1st-generation High Optical Perfor...,1HOPSAT-TD,NR,USA,Hera Systems,Commercial,Earth Observation,Infrared Imaging,LEO,Non-Polar Inclined,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAC AIS-Sat1 (Kelpie 1),AAC AIS-Sat1 (Kelpie 1),United Kingdom,United Kingdom,AAC Clyde Space,Commercial,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aalto-1,Aalto-1,Finland,Finland,Aalto University,Civil,Technology Development,NaN,LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAt-4,AAt-4,Denmark,Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"ABS-2 (Koreasat-8, ST-3)",ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ABS-2A,ABS-2A,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ABS-3A,ABS-3A,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"ABS-4 (ABS-2i, MBSat, Mobile Broadcasting Sate...",ABS-4,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"ABS-6 (ABS-1, LMI-1, Lockheed Martin-Intersput...",ABS-6,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Adelis-Sampson 1,Adelis-Sampson 1,NR,Israel,Asher Space Research Institute at Technion/Isr...,Government,Technology Development,NaN,LEO,Sun-Synchronous,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
ucs_sats_messy.shape

(7560, 68)

In [4]:
ucs_sats_messy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 68 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Name of Satellite, Alternate Names  7560 non-null   object 
 1   Current Official Name of Satellite  7560 non-null   object 
 2   Country/Org of UN Registry          7559 non-null   object 
 3   Country of Operator/Owner           7560 non-null   object 
 4   Operator/Owner                      7560 non-null   object 
 5   Users                               7560 non-null   object 
 6   Purpose                             7560 non-null   object 
 7   Detailed Purpose                    1254 non-null   object 
 8   Class of Orbit                      7560 non-null   object 
 9   Type of Orbit                       6909 non-null   object 
 10  Longitude of GEO (degrees)          7557 non-null   float64
 11  Perigee (km)                        7553 no

In [5]:
ucs_sats_messy.isnull().sum()

Name of Satellite, Alternate Names       0
Current Official Name of Satellite       0
Country/Org of UN Registry               1
Country of Operator/Owner                0
Operator/Owner                           0
                                      ... 
Unnamed: 63                           7075
Unnamed: 64                           7075
Unnamed: 65                           7075
Unnamed: 66                           7073
Unnamed: 67                           7075
Length: 68, dtype: int64

In [6]:
ucs_sats_messy.drop( columns=['Unnamed: 28', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49',
       'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53',
       'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57',
       'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61',
       'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65',
       'Unnamed: 66', 'Unnamed: 67',' Dry Mass (kg.) ', 'Power (watts)'], inplace=True)

In [7]:
print(f"Dtypes: \n\r{ucs_sats_messy.dtypes}\n\r")
print(f"Columns: \n\r{ucs_sats_messy.columns}\n\r")

Dtypes: 
Name of Satellite, Alternate Names     object
Current Official Name of Satellite     object
Country/Org of UN Registry             object
Country of Operator/Owner              object
Operator/Owner                         object
Users                                  object
Purpose                                object
Detailed Purpose                       object
Class of Orbit                         object
Type of Orbit                          object
Longitude of GEO (degrees)            float64
Perigee (km)                           object
Apogee (km)                            object
Eccentricity                          float64
Inclination (degrees)                 float64
Period (minutes)                       object
Launch Mass (kg.)                      object
Date of Launch                         object
Expected Lifetime (yrs.)              float64
Contractor                             object
Country of Contractor                  object
Launch Site              

We need to clean up the Users column, there are duplicates with minor differences that need merged together.

In [8]:
print(f"{ucs_sats_messy['Users'].info()}\n\r")

<class 'pandas.core.series.Series'>
RangeIndex: 7560 entries, 0 to 7559
Series name: Users
Non-Null Count  Dtype 
--------------  ----- 
7560 non-null   object
dtypes: object(1)
memory usage: 59.2+ KB
None



In [9]:
print(f"Users: \n\r{ucs_sats_messy['Users'].unique()}\n\r")

Users: 
['Commercial' 'Civil' 'Government' 'Military' 'Military/Commercial'
 'Government/Military' 'Military/Government' 'Government/Civil'
 'Military/Civil' 'Commercial/Civil' 'Civil/Commercial'
 'Government/Commercial' 'Commercial/Government'
 'Government/Commercial/Military' 'Civil/Government' 'Civil/Military'
 'Commercial ' 'Commercial/Military' 'Government ' 'Military ']



In [10]:
print(f"{ucs_sats_messy.loc[ucs_sats_messy['Users'] == 'Commercial']}")

                     Name of Satellite, Alternate Names  \
0     1HOPSAT-TD (1st-generation High Optical Perfor...   
1                               AAC AIS-Sat1 (Kelpie 1)   
4                              ABS-2 (Koreasat-8, ST-3)   
5                                                ABS-2A   
6                                               ABS-3A    
...                                                 ...   
7550                                Zhuhai 1-03 (OVS-3)   
7551                                Zhuhai 1-04 (OHS-1)   
7552                                Zhuhai 1-05 (OHS-2)   
7553                                Zhuhai 1-06 (OHS-3)   
7554                                Zhuhai 1-07 (OHS-4)   

     Current Official Name of Satellite Country/Org of UN Registry  \
0                            1HOPSAT-TD                         NR   
1               AAC AIS-Sat1 (Kelpie 1)             United Kingdom   
4                                 ABS-2                         NR   
5          

In [11]:
print(f"{ucs_sats_messy.loc[ucs_sats_messy['Users'] == 'Commercial ']}")

     Name of Satellite, Alternate Names Current Official Name of Satellite  \
2797                             SES-12                             SES-12   

     Country/Org of UN Registry Country of Operator/Owner Operator/Owner  \
2797                         NR                Luxembourg       SES S.A.   

            Users         Purpose Detailed Purpose Class of Orbit  \
2797  Commercial   Communications              NaN            GEO   

     Type of Orbit  ...  NORAD Number  \
2797           NaN  ...         43488   

                                               Comments  \
2797  Carries plasma thrusters mounted to articulati...   

     Source Used for Orbital Data  \
2797                     JM/10_18   

                                                 Source  Source.1 Source.2  \
2797  https://spaceflightnow.com/2018/06/04/multi-mi...       NaN      NaN   

     Source.3 Source.4  Source.5 Source.6  
2797      NaN      NaN       NaN      NaN  

[1 rows x 34 columns]


In [12]:
sources = ucs_sats_messy[['Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2', 'Source.3', 'Source.4', 'Source.5', 'Source.6']]

sources

,Source Used for Orbital Data,Source,Source.1,Source.2,Source.3,Source.4,Source.5,Source.6
0,JMSatcat/3_20,https://spaceflightnow.com/2019/12/11/indias-5...,https://www.herasys.com/,NaN,NaN,NaN,NaN,NaN
1,JMSatcat/9_23,https://www.aac-clyde.space/articles/aac-clyde...,NaN,NaN,NaN,NaN,NaN,NaN
2,JMSatcat/10_17,https://directory.eoportal.org/web/eoportal/sa...,NaN,http://www.planet4589.org/space/log/satcat.txt,NaN,NaN,NaN,NaN
3,Space50,http://spaceflightnow.com/2016/04/26/soyuz-bla...,NaN,http://space50.org/objekt.php?mot=2016-025E&ja...,NaN,NaN,NaN,NaN
4,ZARYA,http://www.absatellite.net/satellite-fleet/?sa...,NaN,http://www.zarya.info/Diaries/Launches/Launche...,http://www.absatellite.net/2010/10/13/asia-bro...,http://www.spacenews.com/article/satellite-tel...,NaN,NaN
...,...,...,...,...,...,...,...,...
7555,www.spacedebris.net 12/12,http://www.spaceflightnow.com/news/n1201/09lon...,NaN,https://spacenews.com/china-launches-five-comm...,NaN,NaN,NaN,NaN
7556,Space50,http://spaceflightnow.com/2016/05/31/long-marc...,NaN,https://www.planet4589.org/space/log/satcat.txt,http://space50.org/objekt.php?mot=2016-033A&ja...,NaN,NaN,NaN
7557,ZARYA,https://spaceflightnow.com/2020/07/25/china-la...,NaN,http://www.lib.cas.cz/space.40/2011/079A.HTM,NaN,NaN,NaN,NaN
7558,JMSatcat/1_22,https://spaceflightnow.com/2021/11/15/japanese...,NaN,https://www.planet4589.org/space/log/satcat.txt,NaN,NaN,NaN,NaN


In [13]:
sources.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Source Used for Orbital Data  6637 non-null   object
 1   Source                        3286 non-null   object
 2   Source.1                      725 non-null    object
 3   Source.2                      1832 non-null   object
 4   Source.3                      1126 non-null   object
 5   Source.4                      729 non-null    object
 6   Source.5                      553 non-null    object
 7   Source.6                      504 non-null    object
dtypes: object(8)
memory usage: 472.6+ KB


I would like to drop the sources columns from the main csv but I want to maintain a usable list of this data incase I need it in the future. Output source data to a new csv with noradid added for a primary key for later comparison/cross referencing.

In [14]:
norad_data = ucs_sats_messy['NORAD Number']

sources.insert(0, 'NORAD Number', norad_data)
sources.to_csv('./../data/clean/ucs_sources.csv', index=False)

In [15]:
sources.head(10)

,NORAD Number,Source Used for Orbital Data,Source,Source.1,Source.2,Source.3,Source.4,Source.5,Source.6
0,44859,JMSatcat/3_20,https://spaceflightnow.com/2019/12/11/indias-5...,https://www.herasys.com/,NaN,NaN,NaN,NaN,NaN
1,55107,JMSatcat/9_23,https://www.aac-clyde.space/articles/aac-clyde...,NaN,NaN,NaN,NaN,NaN,NaN
2,42775,JMSatcat/10_17,https://directory.eoportal.org/web/eoportal/sa...,NaN,http://www.planet4589.org/space/log/satcat.txt,NaN,NaN,NaN,NaN
3,41460,Space50,http://spaceflightnow.com/2016/04/26/soyuz-bla...,NaN,http://space50.org/objekt.php?mot=2016-025E&ja...,NaN,NaN,NaN,NaN
4,39508,ZARYA,http://www.absatellite.net/satellite-fleet/?sa...,NaN,http://www.zarya.info/Diaries/Launches/Launche...,http://www.absatellite.net/2010/10/13/asia-bro...,http://www.spacenews.com/article/satellite-tel...,NaN,NaN
5,41588,Estimated,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,40424,NaN,http://www.absatellite.net/satellite-fleet/?sa...,NaN,http://spaceflightnow.com/2015/03/17/innovativ...,NaN,NaN,NaN,NaN
7,28184,JM/12_08,http://www.absatellite.net/satellite-fleet/?sa...,NaN,NaN,http://space.skyrocket.de/doc_sdat/mbsat-1.htm,NaN,NaN,NaN
8,25924,JM/12_08,http://www.washingtonpost.com/wp-dyn/articles/...,NaN,http://nssdc.gsfc.nasa.gov/spacewarn/spx551.html,http://www.interpsputnik.com/1mi1chars.shtml,http://www.intersputnik.com/press_270999.htm,http://www.lockheedmartin.com/content/dam/lock...,NaN
9,47937,JMSatcat/5_21,https://asri.technion.ac.il/node/219,https://spacenews.com/soyuz-2-rideshare-launch...,NaN,NaN,NaN,NaN,NaN


In [16]:
ucs_sats_messy.drop(columns=['Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2', 'Source.3', 'Source.4', 'Source.5', 'Source.6'], inplace=True)

ucs_sats_messy.head()

,"Name of Satellite, Alternate Names",Current Official Name of Satellite,Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,...,Launch Mass (kg.),Date of Launch,Expected Lifetime (yrs.),Contractor,Country of Contractor,Launch Site,Launch Vehicle,COSPAR Number,NORAD Number,Comments
0,1HOPSAT-TD (1st-generation High Optical Perfor...,1HOPSAT-TD,NR,USA,Hera Systems,Commercial,Earth Observation,Infrared Imaging,LEO,Non-Polar Inclined,...,22,12/11/2019,0.5,Hera Systems,USA,Satish Dhawan Space Centre,PSLV,2019-089H,44859,Pathfinder for planned earth observation const...
1,AAC AIS-Sat1 (Kelpie 1),AAC AIS-Sat1 (Kelpie 1),United Kingdom,United Kingdom,AAC Clyde Space,Commercial,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,...,4,1/3/2023,NaN,AAC Clyde Space,Sweden/UK/USA/Netherlands,Cape Canaveral,Falcon 9,2023-001DC,55107,Provide AIS information to Orbcomm.
2,Aalto-1,Aalto-1,Finland,Finland,Aalto University,Civil,Technology Development,NaN,LEO,Sun-Synchronous,...,5,6/23/2017,2.0,Aalto University,Finland,Satish Dhawan Space Centre,PSLV,2017-036L,42775,Technology development and education.
3,AAt-4,AAt-4,Denmark,Denmark,University of Aalborg,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,...,1,4/25/2016,NaN,University of Aalborg,Denmark,Guiana Space Center,Soyuz-2.1a,2016-025E,41460,Carries AIS system.
4,"ABS-2 (Koreasat-8, ST-3)",ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,...,"6,330",2/6/2014,15.0,Space Systems/Loral,USA,Guiana Space Center,Ariane 5 ECA,2014-006A,39508,"32 C-band, 51 Ku-band, and 6 Ka-band transpond..."


In [18]:
ucs_sats_messy.columns

Index(['Name of Satellite, Alternate Names',
       'Current Official Name of Satellite', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Date of Launch', 'Expected Lifetime (yrs.)',
       'Contractor', 'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments'],
      dtype='object')

Save the cleaned data to a new csv for use after cleanup.

In [17]:
ucs_sats_messy.to_csv('./../data/clean/ucs_cleaned.csv', index=False)